# Clustering Crypto

In [24]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [25]:
# Load the crypto_data.csv dataset.
file = 'crypto_data.csv'
df = pd.read_csv(file)
df.dtypes


Unnamed: 0          object
CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [26]:
# Keep all the cryptocurrencies that are being traded.
df = df[df['IsTrading'] == True]


In [27]:
# Keep all the cryptocurrencies that have a working algorithm.
df['Algorithm'] = df['Algorithm'].dropna()


In [28]:
# Remove the "IsTrading" column. 
df = df.drop(columns = 'IsTrading')

In [29]:
# Remove rows that have at least 1 null value.
df_nona = df.dropna()
df_nona.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [30]:
# Keep the rows where coins are mined.
df = df[df['TotalCoinsMined'] > 0]
df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [34]:
# Create a new DataFrame that holds only the cryptocurrencies names.
df_names = df['CoinName'].values
df_names

array(['42 Coin', '404Coin', 'EliteCoin', 'Bitcoin', 'Ethereum',
       'Litecoin', 'Dash', 'Monero', 'Ethereum Classic', 'ZCash',
       'Bitshares', 'DigiByte', 'BitcoinDark', 'PayCoin', 'ProsperCoin',
       'KoboCoin', 'Spreadcoin', 'Argentum', 'Aurora Coin', 'BlueCoin',
       'MyriadCoin', 'MoonCoin', 'ZetaCoin', 'SexCoin', 'Quatloo',
       'EnergyCoin', 'QuarkCoin', 'Riecoin', 'Digitalcoin ', 'BitBar',
       'Catcoin', 'CryptoBullion', 'CannaCoin', 'CryptCoin', 'CasinoCoin',
       'Diamond', 'Verge', 'DevCoin', 'EarthCoin', 'E-Gulden',
       'Einsteinium', 'Emerald', 'Exclusive Coin', 'FlutterCoin',
       'Franko', 'FeatherCoin', 'GrandCoin', 'GlobalCoin', 'GoldCoin',
       'HoboNickels', 'HyperStake', 'Infinite Coin', 'IOCoin', 'IXcoin',
       'KrugerCoin', 'LuckyCoin', 'Litebar ', 'MaxCoin', 'MegaCoin',
       'MediterraneanCoin', 'MintCoin', 'MinCoin', 'MazaCoin',
       'Nautilus Coin', 'NavCoin', 'NobleCoin', 'Namecoin', 'NyanCoin',
       'OpalCoin', 'Orbitcoin', 'P

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df = df.drop(columns = 'CoinName')

In [ ]:
df = df.drop(columns = 'Unnamed: 0')
df.head()

In [ ]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(df, columns=['Algorithm', 'ProofType'], dtype=float)
X

In [ ]:
# Standardize the data with StandardScaler().
scaler=StandardScaler()
scaled_df = scaler.fit_transform(X)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)


In [35]:
# Create a DataFrame with the three principal components.
df_pca = pca.fit_transform(scaled_df)
df_pca = pd.DataFrame(df_pca, columns=["PC 1", "PC 2", "PC 3"], index=df_names)
df_pca.head()

,PC 1,PC 2,PC 3
42 Coin,-0.328887,0.997172,-0.620896
404Coin,-0.312220,0.997236,-0.621441
EliteCoin,2.320250,1.528785,-0.737473
Bitcoin,-0.144358,-1.323201,0.201174
Ethereum,-0.162431,-2.062860,0.407853


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1,11))

for i in k:
    km = KMeans(n_clusters=i, random_state=98)
    km.fit(df_pca)
    inertia.append(km.inertia_)
    
elbow_data = {'k': k, 'inertia': inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x='k', y='inertia', xticks = k, title='elbow curve')

Running K-Means with `k=4`

In [36]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=98)

# Fit the model
model.fit(df_pca)

# Predict clusters
predictions = model.predict(df_pca)
print(predictions)

C:\Users\janze\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


[0 0 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1
 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 1
 1 1 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 0
 1 0 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 1 1 0
 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 1
 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 0 0 0 1 0 0 1
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 1
 0 1 0 1 0 0 3 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0
 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 0 1 1 1 0 1 1 0 0 0 1 1 1 0 0 0
 1 1 0 1 1 1 1 0 3 3 1 1 1 0 3 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 1
 0 0 1 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 1 1 1
 0 0 0 0 1 0 0 1 0 0 1 3 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0
 1 1 1 0 0 0 1 0 1 0 1 0 

In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE


In [ ]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
